In [1]:
import pandas as pd
import numpy as np

TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'
VOCAB_SIZE = 2500

In [2]:
train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype= int)
test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype= int)

In [3]:
test_data.shape

(113279, 4)

In [4]:
train_data.shape

(262842, 4)

In [5]:
np.unique(train_data[:, 0]).size # Emails in the Training data

4013

In [6]:
np.unique(test_data[:, 0]).size # Emails in the testing data

1724

In [7]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """ function to make a full dataframe from the compact matrix... """
    
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    
    doc_id_names = np.unique(sparse_matrix[:, 0])
    
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    
    full_matrix.fillna(value=0, inplace=True)
    
    
    for i in range(sparse_matrix.shape[0]):
        
        doc_nr = sparse_matrix[i][doc_idx]
        
        word_id = sparse_matrix[i][word_idx]
        
        label = sparse_matrix[i][cat_idx]
        
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix

In [8]:
full_train_data = make_full_matrix(train_data, VOCAB_SIZE)

full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5795,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5796,0,2,2,0,1,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
5797,0,1,9,1,0,0,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5795,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5796,2,2,0,1,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5797,1,9,1,0,0,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
email_lengths = full_train_features.sum(axis = 1)
email_lengths

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
5795     46
5796     30
5797    228
5798     57
5799    264
Length: 4013, dtype: int64

In [11]:
total_wc=email_lengths.sum()
total_wc

446521

In [12]:
email_lengths.shape

(4013,)

In [13]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths

DOC_ID
1898     82
1899     41
1900    112
1901     66
1903     34
       ... 
5795     46
5796     30
5797    228
5798     57
5799    264
Length: 2767, dtype: int64

In [14]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

253773

In [15]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1246,)

In [16]:
spam_wc = spam_lengths.sum()
spam_wc

192748

In [17]:
print('Number of words in non spam emails   ', nonspam_wc, '\n') 
print('Number of words in  spam emails      ', spam_wc, '\n') 
print('Sum of words in ham and spam mails   ', nonspam_wc + spam_wc)

Number of words in non spam emails    253773 

Number of words in  spam emails       192748 

Sum of words in ham and spam mails    446521


In [18]:
print('email_lengths -->   ',email_lengths.shape[0]) 

print('spam_lengths -->   ', spam_lengths.shape[0]) 

print('ham_lengths -->  ', ham_lengths.shape[0])

email_lengths -->    4013
spam_lengths -->    1246
ham_lengths -->   2767


In [19]:
# which mails are longer?....Spam or Ham?
print('Average number of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average number of words in ham emails {:.0f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average number of words in spam emails 155
Average number of words in ham emails 92


In [20]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens


,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1898,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1900,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1901,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1903,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5795,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5796,2,2,0,1,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5797,1,9,1,0,0,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [21]:
train_ham_tokens.shape

(2767, 2500)

In [22]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) +1  # Laplace smoothing....

summed_ham_tokens

0       5415
1       2610
2       2064
3        902
4       1630
        ... 
2495      12
2496       4
2497      17
2498      16
2499      31
Length: 2500, dtype: int64

In [23]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)

In [24]:
prob_tokens_nonspam.sum()

1.0

In [25]:
prob_tokens_nonspam 

0       0.021130
1       0.010184
2       0.008054
3       0.003520
4       0.006360
          ...   
2495    0.000047
2496    0.000016
2497    0.000066
2498    0.000062
2499    0.000121
Length: 2500, dtype: float64

In [26]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,0,0,0,0,0,0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,2,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
train_spam_tokens.shape

(1246, 2500)

In [28]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 

In [29]:
summed_spam_tokens

0       2213
1       1010
2       1366
3       2049
4       1380
        ... 
2495      25
2496       1
2497      15
2498      18
2499       3
Length: 2500, dtype: int64

In [30]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)

In [31]:
prob_tokens_spam

0       0.011334
1       0.005173
2       0.006996
3       0.010494
4       0.007068
          ...   
2495    0.000128
2496    0.000005
2497    0.000077
2498    0.000092
2499    0.000015
Length: 2500, dtype: float64

In [32]:
prob_tokens_spam.sum()

1.0

In [33]:
prob_tokens_all=full_train_features.sum(axis = 0 )/total_wc
prob_tokens_all

0       0.017079
1       0.008103
2       0.007677
3       0.006604
4       0.006737
          ...   
2495    0.000078
2496    0.000007
2497    0.000067
2498    0.000072
2499    0.000072
Length: 2500, dtype: float64

In [34]:
####

In [35]:
TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

In [36]:
test_data.shape

(113279, 4)

In [37]:
full_test_data = make_full_matrix(test_data, nr_words=VOCAB_SIZE)

full_test_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY


In [39]:
X_test.shape

(1724, 2500)

In [40]:
y_test.shape

(1724,)

In [41]:
TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'
np.savetxt(TEST_FEATURE_MATRIX, X_test)
np.savetxt(TEST_TARGET_FILE, y_test)